In [1]:
#import sys
#sys.path.append('dba_utils/') # to access util files 

In [3]:
import numpy as np
import pandas as po
import matplotlib.pyplot as plt

In [3]:
!bash ./utils/build-cython.sh

Compiling dtw.pyx because it changed.
[1/1] Cythonizing dtw.pyx
running build_ext
building 'dtw' extension
creating build
creating build/temp.linux-x86_64-3.7
gcc -pthread -B /home/varun/anaconda3/envs/tf2/compiler_compat -Wl,--sysroot=/ -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include -I/home/varun/anaconda3/envs/tf2/include/python3.7m -c dtw.c -o build/temp.linux-x86_64-3.7/dtw.o
In file included from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/ndarraytypes.h:1832,
                 from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from dtw.c:619:
/home/varun/anaconda3/envs/tf2/lib/python3.7/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_a

In [4]:
from utils.augment import augment_train_set

In [5]:
df = po.read_csv('../data/processed/imputation/edtwbi.csv').sample(frac=1, random_state=42).reset_index(drop=True)

In [6]:
df_0 = df[df['FLAG'] == 0]
df_1 = df[df['FLAG'] == 1]

In [ ]:
# Undersampling
df_0 = df_0.sort_values('num_zeros')[:len(df_1)]

In [8]:
df_undersampled = po.concat([df_1, df_0], axis=0, ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

In [9]:
df_undersampled['FLAG'].value_counts()

1    3615
0    3615
Name: FLAG, dtype: int64

In [10]:
X_undersampled = df_undersampled.drop(['num_zeros', 'FLAG'], axis=1)
y_undersampled = df_undersampled['FLAG']

In [11]:
X_undersampled_train = X_undersampled[:int(0.6*len(X_undersampled))]
y_undersampled_train = y_undersampled[:int(0.6*len(X_undersampled))]

X_undersampled_val   = X_undersampled[int(0.6*len(X_undersampled)):int(0.8*len(X_undersampled))]
y_undersampled_val   = y_undersampled[int(0.6*len(X_undersampled)):int(0.8*len(X_undersampled))]

X_undersampled_test  = X_undersampled[int(0.8*len(X_undersampled)):]
y_undersampled_test  = y_undersampled[int(0.8*len(X_undersampled)):]

In [12]:
X_undersampled_train     = X_undersampled_train.to_numpy()[:, :, np.newaxis]
X_undersampled_val       = X_undersampled_val.to_numpy()[:, :, np.newaxis]
X_undersampled_test      = X_undersampled_test.to_numpy()[:, :, np.newaxis]

In [13]:
y_undersampled_train     = y_undersampled_train.to_numpy()
y_undersampled_val       = y_undersampled_val.to_numpy()
y_undersampled_test      = y_undersampled_test.to_numpy()

In [14]:
# Save undersampled data
with open('../data/processed/training_data/undersampled.npz', 'wb') as f:
    np.savez(f, X_train=X_undersampled_train, y_train=y_undersampled_train, X_val=X_undersampled_val, y_val=y_undersampled_val, X_test=X_undersampled_test, y_test=y_undersampled_test)

In [6]:
X = df.drop(['num_zeros', 'FLAG'], axis=1)
y = df['FLAG']

In [8]:
y.value_counts()

0    20657
1     3615
Name: FLAG, dtype: int64

In [7]:
X_train = X[:int(0.6*len(X))]
y_train = y[:int(0.6*len(X))]

X_val   = X[int(0.6*len(X)):int(0.8*len(X))]
y_val   = y[int(0.6*len(X)):int(0.8*len(X))]

X_test  = X[int(0.8*len(X)):]
y_test  = y[int(0.8*len(X)):]

In [8]:
y_train.value_counts()

0    12381
1     2182
Name: FLAG, dtype: int64

In [9]:
y_val.value_counts()

0    4121
1     733
Name: FLAG, dtype: int64

In [10]:
y_test.value_counts()

0    4155
1     700
Name: FLAG, dtype: int64

In [11]:
X_train     = X_train.to_numpy()[:, :, np.newaxis]
X_val       = X_val.to_numpy()[:, :, np.newaxis]
X_test      = X_test.to_numpy()[:, :, np.newaxis]

In [12]:
y_train     = y_train.to_numpy()
y_val       = y_val.to_numpy()
y_test      = y_test.to_numpy()

In [13]:
# Save raw data
with open('../data/processed/training_data/standard.npz', 'wb') as f:
    np.savez(f, X_train=X_train, y_train=y_train, X_val=X_val, y_val=y_val, X_test=X_test, y_test=y_test)

In [14]:
N       = 100 

In [15]:
classes, classes_counts = np.unique(y_train, return_counts=True)

In [16]:
classes

array([0, 1])

In [17]:
# augment the dataset
X_train_synth, y_train_synth = augment_train_set(X_train, y_train, classes, N, limit_N = False, weights_method_name='as', distance_algorithm='dtw')

79 [00:36<31:04,  6.50it/s]
  3%|▎         | 407/12379 [00:58<28:32,  6.99it/s]Process ForkPoolWorker-3:
Process ForkPoolWorker-11:

  0%|          | 1/12380 [28:56<5971:43:53, 1736.67s/it]Process ForkPoolWorker-6:
Process ForkPoolWorker-16:

Process ForkPoolWorker-8:
Process ForkPoolWorker-7:
Process ForkPoolWorker-10:
Process ForkPoolWorker-9:
Process ForkPoolWorker-5:
Process ForkPoolWorker-2:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/varun/anaconda3/envs/tf2/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/varun/anaconda3/envs/tf2/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/varun/anaconda3/envs/tf2/lib/python3.7/multiprocessing/process.p

KeyboardInterrupt: 